<a href="https://colab.research.google.com/github/Ansil-bayan/MachineTranslation_en_to_ml/blob/main/MachineTranslation_en_to_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch peft datasets sacrebleu evaluate

In [2]:
from transformers import MBartForConditionalGeneration, MBart50Tokenizer, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments

In [3]:
from datasets import load_dataset
import evaluate

In [27]:
import numpy as np
import torch, os
os.environ["WANDB_DISABLED"] = "true"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
raw_dataset = load_dataset("ai4bharat/samanantar","ml",split="train[:1000]", trust_remote_code=True)

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'ai4bharat/samanantar' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
ERROR:datasets.load:`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'ai4bharat/samanantar' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able t

In [6]:
dataset = raw_dataset.take(1000)
dataset

Dataset({
    features: ['idx', 'src', 'tgt'],
    num_rows: 1000
})

In [7]:
tokenized_dataset = dataset.train_test_split(test_size=0.1)

In [8]:
train_dataset =tokenized_dataset["train"]
eval_dataset = tokenized_dataset["test"]
print(train_dataset[0])
print(eval_dataset[0])

{'idx': 474, 'src': "Thats the film's script.", 'tgt': 'തിരക്കഥയാണ് ഈ സിനിമയുടെ ജീവൻ.'}
{'idx': 422, 'src': 'Women or men?', 'tgt': 'ആണോ പുരുഷനോ സ്ത്രീയോ?'}


In [9]:
model_name = "facebook/mbart-large-50-many-to-many-mmt"
model = MBartForConditionalGeneration.from_pretrained(model_name)

In [10]:
tokenizer = MBart50Tokenizer.from_pretrained(model_name)

In [11]:
def preprocess_function(a):           #to convert train dataset and eval dataset into token numbers
  inputs = [i for i in a['src']]
  targets = [i for i in a['tgt']]

  model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

  labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length").input_ids
  labels = [[-100 if token == tokenizer.pad_token_id else token for token in label] for label in labels]

  model_inputs["labels"] = labels
  return model_inputs

tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [12]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy= "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    fp16=True,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=10,
    generation_max_length=128,
    report_to="none"

)

In [13]:
metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
  preds,labels = eval_preds

  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

  labels = np.where(labels!=-100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  result = metric.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])

  return result

In [14]:
trainer = Seq2SeqTrainer(
    model=model,
    args= training_args,
    train_dataset= tokenized_train_dataset,
    eval_dataset= tokenized_eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-3229105591.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [15]:
save_directory = "/kaggle/new_model/saved_model"

os.makedirs(save_directory, exist_ok=True)

In [16]:
trainer.train()

Epoch,Training Loss,Validation Loss,Score,Counts,Totals,Precisions,Bp,Sys Len,Ref Len
1,3.272500,2.889647,3.322906,"[203, 38, 10, 2]","[738, 638, 538, 441]","[27.506775067750677, 5.956112852664577, 1.858736059479554, 0.45351473922902497]",0.969315,738,761
2,2.638200,2.804257,2.961571,"[211, 36, 9, 2]","[808, 708, 608, 511]","[26.113861386138613, 5.084745762711864, 1.480263157894737, 0.3913894324853229]",1.000000,808,761


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3854: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Score,Counts,Totals,Precisions,Bp,Sys Len,Ref Len
1,3.272500,2.889647,3.322906,"[203, 38, 10, 2]","[738, 638, 538, 441]","[27.506775067750677, 5.956112852664577, 1.858736059479554, 0.45351473922902497]",0.969315,738,761
2,2.638200,2.804257,2.961571,"[211, 36, 9, 2]","[808, 708, 608, 511]","[26.113861386138613, 5.084745762711864, 1.480263157894737, 0.3913894324853229]",1.000000,808,761
3,2.283900,2.801660,2.909508,"[217, 36, 8, 2]","[805, 705, 605, 508]","[26.956521739130434, 5.1063829787234045, 1.322314049586777, 0.3937007874015748]",1.000000,805,761


TrainOutput(global_step=171, training_loss=2.9214042338014345, metrics={'train_runtime': 577.3402, 'train_samples_per_second': 4.677, 'train_steps_per_second': 0.296, 'total_flos': 731406296678400.0, 'train_loss': 2.9214042338014345, 'epoch': 3.0})

In [17]:
trainer.save_model(save_directory)

In [18]:
save_directory = './finetuned-MBart50-en-ml'

# Save model and tokenizer
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"Model saved to {save_directory}")

Model saved to ./finetuned-MBart50-en-ml


In [28]:
# @title Translate {"display-mode":"form"}

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Load your model and tokenizer
model_name =save_directory
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Example English sentence to translate
English = "Thats the film's script" # @param {"type":"string"}

# Tokenize the input sentence
inputs = tokenizer(English, return_tensors="pt").to(device)

# Generate translation
with torch.no_grad():
    outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)

# Decode the generated tokens
translated_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"English: {English}")
print(f"Mal Translation: {translated_sentence}")

English: Thats the film's script
Mal Translation: അതാണ് ചിത്രത്തിന് റെ തിരക്കഥ.
